# LCEL Runnable 프로토콜

## 1. Runnable 프로토콜 소개

### Runnable이란?
- **Runnable**은 LangChain의 핵심 인터페이스로, 작업의 단위(unit of work)를 나타내는 프로토콜입니다
- 호출(invoke), 배치(batch), 스트리밍(stream), 변환(transform), 구성(compose)이 가능한 표준 인터페이스입니다
- LangChain의 모든 주요 구성요소(LLM, 프롬프트, 출력 파서, 리트리버 등)는 Runnable 프로토콜을 구현합니다


## 2. Runnable의 주요 메서드

### 핵심 메서드들
| 메서드 | 설명 | 사용 예시 |
|--------|------|-----------|
| `invoke()` | 단일 입력을 받아 출력 생성 | `runnable.invoke(input)` |
| `batch()` | 여러 입력을 병렬로 처리 | `runnable.batch([input1, input2])` |
| `stream()` | 출력을 스트리밍으로 생성 | `runnable.stream(input)` |
| `astream_events()` | 이벤트 스트리밍 (고급) | `runnable.astream_events(input)` |

### 입력/출력 타입 (컴포넌트별)
| 컴포넌트 | 입력 타입 | 출력 타입 |
|----------|-----------|-----------|
| **Prompt** | 딕셔너리 객체 | PromptValue |
| **ChatModel** | 문자열, 메시지 리스트, PromptValue | ChatMessage |
| **LLM** | 문자열, 메시지 리스트, PromptValue | 문자열 |
| **OutputParser** | LLM 또는 ChatModel의 출력 | 파서에 따라 다름 |
| **Retriever** | 문자열 | Document 리스트 |
| **Tool** | 문자열 또는 객체 | 도구에 따라 다름 |


**실습 준비 - 기본 설정**

In [ ]:
# 필요한 라이브러리 가져오기
from langchain_ollama import ChatOllama
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnableParallel

# LLM 모델 초기화
llm = ChatOllama(
    model="midm-2.0-base-instruct-q5_k_m",
    temperature=0.7,
)

print("모든 모듈이 성공적으로 로드되었습니다!")


### 기본 Runnable 실습

**invoke() 메서드 - 단일 입력 처리**

In [ ]:
# 1. 프롬프트 템플릿 생성 (Runnable)
prompt = PromptTemplate.from_template("다음 주제에 대해 한줄로 간단히 설명해주세요: {topic}")

# 2. 출력 파서 생성 (Runnable)
output_parser = StrOutputParser()

# Langchain의 각 컴포넌트는 Runnable 프로토콜을 구현합니다
# invoke 메서드를 사용하여 각 단계별로 실행
input_data = {"topic": "랭체인"}

In [ ]:
# 1단계: 프롬프트 생성
formatted_prompt = prompt.invoke(input_data)
print(formatted_prompt)

In [ ]:
# 2단계: LLM 실행
llm_response = llm.invoke(formatted_prompt)
print(llm_response)

In [ ]:
# 3단계: 출력 파싱
final_output = output_parser.invoke(llm_response)
print(final_output)

**체인 구성 - Pipe 연산자 (|) 사용**
- LCEL의 핵심 기능: Pipe 연산자(`|`)를 사용한 체인 구성
- 여러 Runnable을 연결하여 하나의 체인으로 만들 수 있습니다


In [ ]:
# 방법 1: Pipe 연산자 사용
chain = prompt | llm | output_parser

# 체인을 하나의 Runnable로 사용
result = chain.invoke({"topic": "머신러닝"})
print(result)

### RunnableSequence란?
- 여러 Runnable을 순차적으로 실행하는 체인을 명시적으로 생성하는 클래스
- Pipe 연산자(|)를 사용하는 것과 동일한 결과를 제공
- 복잡한 체인 구성에서 유용

In [ ]:
# 방법 2: RunnableSequence 클래스 사용
from langchain_core.runnables import RunnableSequence

sequence_chain = RunnableSequence(prompt, llm, output_parser)

# 동일한 결과
result2 = sequence_chain.invoke({"topic": "딥러닝"})
print("RunnableSequence 결과:")
print(result2)


**batch() 메서드**
- 여러 입력들 (목록)에 대해 체인을 호출 할 수 있습니다.

In [ ]:
# batch 메서드를 사용하여 여러 입력을 한 번에 처리
topics = [
    {"topic": "블록체인"},
    {"topic": "양자컴퓨팅"},
    {"topic": "사물인터넷"}
]

batch_results = chain.batch(topics)

for i, (topic, result) in enumerate(zip(topics, batch_results)):
    print(f"\n{i+1}. {topic['topic']}:")
    print(result)


**stream() 메서드**
- LLM의 출력을 실시간으로 스트리밍하여 받을 수 있는 메서드

In [ ]:
# stream 메서드를 사용하여 실시간으로 출력 스트리밍
print("스트리밍 출력 (실시간으로 토큰이 생성됩니다):")

for chunk in chain.stream({"topic": "자연어처리"}):
    print(chunk, end="", flush=True)

### RunnableParallel
- 여러 Runnable을 병렬로 실행하는 체인을 생성할 수 있다.

In [ ]:
# 다양한 관점의 프롬프트 생성
tech_prompt = PromptTemplate.from_template("기술적 관점에서 {topic}에 대해 한줄로 설명해주세요")
military_prompt = PromptTemplate.from_template("군사적 역량 관점에서 {topic}의 활용 방안을 한줄로 설명해주세요")
social_prompt = PromptTemplate.from_template("사회적 영향 관점에서 {topic}에 대해 한줄로 설명해주세요")

# 병렬 체인 구성
parallel_chain = RunnableParallel(
    technical=tech_prompt | llm | output_parser,
    military=military_prompt | llm | output_parser,
    social=social_prompt | llm | output_parser
)

# 병렬 실행
topic_input = {"topic": "인공지능"}
results = parallel_chain.invoke(topic_input)

print("1. 기술적 관점:")
print(results["technical"])
print("2. 군사적 역량 관점:")
print(results["military"])
print("3. 사회적 영향 관점:")
print(results["social"])

### RunnableLambda 
- 커스텀 함수를 Runnable로 변환할 수 있다.
- 일반적인 Python 함수를 LangChain 체인에 포함시킬 수 있도록 감싸는 래퍼(wrapper) 기능

In [ ]:
# 1. 간단한 변환 함수들
# 텍스트에 접두사 추가
def add_prefix(text: str) -> str:
    return f"📝 한줄설명: {text}"

# 함수들을 Runnable로 변환
prefix_runnable = RunnableLambda(add_prefix)

# 테스트
sample_text = "인공지능은 미래 기술의 핵심입니다"

print("원본 텍스트:", sample_text)
print("접두사 추가:", prefix_runnable.invoke(sample_text))


Runnable 인터페이스로 만들어지기 때문에, chain으로 사용할 수 있습니다.

In [ ]:
enhanced_chain = (
    prompt 
    | llm 
    | output_parser 
    | prefix_runnable  # 커스텀 함수를 체인에 추가
)

result = enhanced_chain.invoke({"topic": "국방부"})
print("후처리가 포함된 체인 결과:")
print(result)